In [1]:
import os
import yaml
import urllib
from PIL import Image
from enum import Enum
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import xml.etree.cElementTree as ET
import glob
import argparse
import numpy as np
import json
import numpy
import cv2
from collections import OrderedDict
import scipy.misc
from skimage import measure   
from shapely.geometry import Polygon, MultiPolygon, MultiPoint
import random
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import shutil
import pickle

WORKING_DIRECTORY = '/media/dean/datastore/datasets/darknet_evaluate'
COCO_DIRECTORY = os.path.join(WORKING_DIRECTORY, 'data', 'coco')
BDD10K_COCO_ANNOTATIONS_FILE = os.path.join(COCO_DIRECTORY,'annotations' , 'bdd10k_instances_val2014.json')
IMAGES_DIRECTORY = os.path.join(COCO_DIRECTORY, 'images','train2014')
WEIGHTS_DIRECTORY = os.path.join(WORKING_DIRECTORY,'trained_weights')
LABELS_DIRECTORY = os.path.join(COCO_DIRECTORY, 'labels','train2014')
CATEGORY_NAMES = os.path.join(WORKING_DIRECTORY, 'data', 'coco.bdd100k.names')

In [2]:
bdd10k_coco = COCO(BDD10K_COCO_ANNOTATIONS_FILE)
ann_ids = bdd10k_coco.getAnnIds()
print(len(ann_ids))

loading annotations into memory...
Done (t=2.62s)
creating index...
index created!
185578


In [ ]:
# Grab batches and sort checkpoints
d = {}
weights_files = glob.glob(os.path.join(WEIGHTS_DIRECTORY, '*.weights'))

for fpath in weights_files:
    fname = os.path.split(fpath)[-1]
    batches = fname[15:].split('.weights')[0]
    d[fname] = int(batches)
sorted_weights = OrderedDict(sorted(d.items(), key=lambda kv: kv[1]))
bdd_weights = OrderedDict()

for weights_file in sorted_weights.keys():
    d = {}
    d['batches'] = sorted_weights[weights_file]
    d['map_results_file'] = os.path.join(WEIGHTS_DIRECTORY, weights_file+'.txt')
    map_results_file = d['map_results_file']
    
    if not os.path.exists(d['map_results_file']+'.backup'):
        # Create annotations if labels directory not found
        if not os.path.exists(os.path.join(COCO_DIRECTORY,'labels/train2014/manifast.txt')):
            #!java -jar $WORKING_DIRECTORY/data/cocotoyolo.jar "{BDD10K_COCO_ANNOTATIONS_FILE}" "{IMAGES_DIRECTORY}/" "all" "{COCO_DIRECTORY}/labels/train2014"
            yolo_convert_output = os.path.join(COCO_DIRECTORY, 'labels','convert2yolo_results.txt')
            !python3 $WORKING_DIRECTORY/convert2Yolo/example.py --datasets COCO --img_path "{IMAGES_DIRECTORY}" --label "{BDD10K_COCO_ANNOTATIONS_FILE}" --convert_output_path "{LABELS_DIRECTORY}" --img_type [".jpg"] --manipast_path $LABELS_DIRECTORY --cls_list_file $CATEGORY_NAMES | tee -a $yolo_convert_output
        
        outfile = !cd $WORKING_DIRECTORY && ./darknet detector map cfg/bdd100k.data cfg/yolov3-bdd100k.cfg $WEIGHTS_DIRECTORY/$weights_file | tee -a $map_results_file
        with open(d['map_results_file']+'.backup',"w+") as f:
            outfile = '\n'.join(outfile)
            f.write(outfile)
    
    #Get mAP results from file
    class_stats = []
    map_stats = {}
    with open(d['map_results_file']+'.backup') as openfile:
        for line in openfile:
            for part in line.split():
                if 'class_id' in part:
                    class_map = {}
                    id_tokens = line.split('class_id =')[1].split(',')
                    class_map['class_id'] = int(id_tokens[0].strip())

                    name_tokens = line.split('name =')[1].split(',')
                    class_map['class_name'] = name_tokens[0]

                    if 'ap =' in line:
                        ap_tokens = line.split('ap =')[1].split(',')
                        class_map['class_ap'] = ap_tokens[0].split('%')[0].strip()

                    class_stats.append(class_map)
                elif '(mAP)' in part:
                    tokens = line.split('(mAP) =')[1]
                    map_stats['mean_avg_precision'] = tokens.split()[2].strip(',')
                elif 'Detection' in part:
                    tokens = line.split(':')[1]
                    total_detection_time = str(''.join(tokens)).strip()
                    map_stats['total_detection_time'] = total_detection_time
    map_stats['class_stats'] = class_stats
    print(map_stats,'\n\n') 
    bdd_weights[weights_file] = map_stats

    
    
# Make sure Dictionary is made properly
print(bdd_weights)

COCO Parsing:  |----------------------------------------| 0.1% (190/185578)  Complete

COCO Parsing:  |----------------------------------------| 0.2% (285/185578)  Complete

COCO Parsing:  |----------------------------------------| 0.2% (380/185578)  Complete

COCO Parsing:  |----------------------------------------| 0.3% (475/185578)  Complete

COCO Parsing:  |----------------------------------------| 0.3% (570/185578)  Complete

...

COCO Parsing:  |████████████████████████████████████████| 100.0% (185578/185578)  Complete��████████████████████████-| 98.4% (182618/185578)  Complete185578)  Complete��███-| 98.4% (182643/185578)  Complete�██████████████████████-| 98.4% (182667/185578)  Completeng:  |███████████████████████████████████████-| 98.4% (182691/185578)  Complete�█-| 98.4% (182692/185578)  Complete7% (183110/185578)  Complete�███████-| 98.7% (183122/185578)  Complete7% (183257/185578)  Complete�███████-| 98.8% (183269/185578)  Complete██████████████████████████-| 98.8% (183293/185578)  Complete�███████████████████████████████████-| 98.8% (183305/185578)  Complete0% (183649/185578)  Complete�███████-| 99.0% (183661/185578)  Complete1% (183845/185578)  Complete�███████-| 99.1% (183857/185578)  Complete2% (184041/185578)  Complete��████████████████-| 99.2% (184065/185578)  Complete�███████████████████████████████████-| 99.2% (184089/185578)  Complete% (184113/185578)  Complete███████-| 99.2% (184125/185578)  Co

YOLO Generating:|███████████████████████-----------------| 59.2% (5916/10

YOLO Generating:|████████████████████████████████████████| 100.0% (10000/10000)  Complete�██████████████████████-----------------| 59.8% (5976/10000)  Completeing:|███████████████████████-----------------| 59.9% (5993/10000)  Complete0.1% (6007/10000)  Complete�█████████████████████----------------| 60.7% (6066/10000)  Complete000)  Completeting:|████████████████████████----------------| 61.1% (6112/10000)  Complete█████████████████----------------| 61.4% (6141/10000)  Completee��█████████████████████----------------| 61.9% (6187/10000)  Complete��████████████----------------| 62.2% (6216/10000)  Complete��███----------------| 62.5% (6245/10000)  Complete�████████---------------| 62.8% (6276/10000)  Complete��██████████████████---------------| 63.0% (6305/10000)  CompleteGenerating:|█████████████████████████---------------| 63.3% (6334/10000)  Completeomplete Generating:|███████████████████████████-------------| 69.5% (6945/10000)  Completemplete████████------------| 70.3% (7030/10000)

YOLO Saving:   |█████████████████████-------------------| 54.5% (5449/10000)  Complete----------------------------------| 1.8% (180/10000)  Complete226/10000)  Complete (272/10000)  Complete------------| 3.0% (299/10000)  Complete--------------------| 3.9% (390/10000)  CompleteCompleteComplete██--------------------------------------| 5.1% (507/10000)  Complete----| 5.5% (551/10000)  Complete Saving:   |██--------------------------------------| 5.8% (577/10000)  Complete----------------| 6.2% (621/10000)  Completelete--------------------------| 7.1% (710/10000)  Complete10000)  Complete0)  Complete--------| 7.8% (779/10000)  Completeaving:   |███-------------------------------------| 8.2% (823/10000)  Completelete------------------------| 8.9% (891/10000)  Complete |███-------------------------------------| 9.1% (913/10000)  CompleteComplete----------------------------| 9.8% (978/10000)  CompleteSaving:   |████------------------------------------| 10.0% (1003/10000)  Complete Complete4%

YOLO Saving:   |████████████████████████████████████████| 100.0% (10000/10000)  Complete█████████------------------| 55.1% (5511/10000)  Complete---| 55.3% (5528/10000)  CompleteSaving:   |██████████████████████------------------| 55.6% (5559/10000)  Complete�████████████------------------| 55.9% (5589/10000)  Complete-----------| 56.1% (5606/10000)  Completee███████████████------------------| 56.7% (5667/10000)  Complete��█------------------| 57.0% (5697/10000)  Complete Complete��█████████████████------------------| 57.5% (5745/10000)  Complete██████████████████████-----------------| 57.8% (5775/10000)  Complete��██████████████████-----------------| 57.9% (5792/10000)  Completeng:   |███████████████████████-----------------| 58.2% (5822/10000)  Complete Complete 58.8% (5881/10000)  Complete98/10000)  Complete---------| 59.3% (5928/10000)  Complete████-----------------| 59.6% (5958/10000)  Complete��-----------------| 59.8% (5975/10000)  Complete��███████████----------------| 60.1% (6

In [ ]:
print(len(bdd_weights))

In [ ]:
import re

class_stats = []
map_stats = {}
with open("/media/dean/datastore/datasets/darknet_evaluate/trained_weights/yolov3-bdd100k_24864.weights.txt.backup") as openfile:
    #Get mAP results from file
    for line in openfile:
        for part in line.split():
            if 'class_id' in part:
                class_map = {}
                id_tokens = line.split('class_id =')[1].split(',')
                class_map['class_id'] = int(id_tokens[0].strip())

                name_tokens = line.split('name =')[1].split(',')
                class_map['class_name'] = name_tokens[0]

                if 'ap =' in line:
                    ap_tokens = line.split('ap =')[1].split(',')
                    class_map['class_ap'] = ap_tokens[0].split('%')[0].strip()

                class_stats.append(class_map)
            elif '(mAP)' in part:
                tokens = line.split('(mAP) =')[1]
                map_stats['mean_avg_precision'] = tokens.split()[2].strip(',')
            elif 'Detection' in part:
                tokens = line.split(':')[1]
                total_detection_time = str(''.join(tokens)).strip()
                map_stats['total_detection_time'] = total_detection_time
    
    map_stats['class_stats'] = class_stats
print(map_stats,'\n\n')
#!cat /media/dean/datastore/datasets/darknet_evaluate/trained_weights/yolov3-bdd100k_24864.weights.txt.backup
    

In [ ]:
import os

class_stats = []
map_stats = {}
class_map = {}
with open("/Users/deanwebb/Desktop/yolov3-bdd100k_28416.weights.txt.backup", 'r') as openfile:
    line = next(openfile)
    while line:
        reg_match = RegExParser(line)

        if reg_match.class_id:
            class_map['class_id'] = reg_match.class_id.group(1)

        if reg_match.class_name:
            class_map['class_name'] = reg_match.class_name.group(1)

        if reg_match.class_ap:
            class_map['class_ap'] = reg_match.class_ap.group(1)

        if reg_match.mean_ap:
            map_stats['mean_avg_precision'] = reg_match.mean_ap.group(1)

        if reg_match.obj_thresh:
            map_stats['obj_thresh'] = reg_match.obj_thresh.group(1)

        if reg_match.precision:
            map_stats['precision'] = reg_match.precision.group(1)

        if reg_match.recall:
            map_stats['recall'] = reg_match.recall.group(1)

        if reg_match.f1:
            map_stats['f1_score'] = reg_match.f1.group(1)

        if reg_match.TP:
            map_stats['TP'] = reg_match.TP.group(1)

        if reg_match.FP:
            map_stats['FP'] = reg_match.FP.group(1)

        if reg_match.FN:
            map_stats['FN'] = reg_match.FN.group(1)

        if reg_match.avg_IoU:
            map_stats['avg_IoU'] = reg_match.avg_IoU.group(1)




    #Get mAP results from file
    for line in openfile:
        for part in line.split():
            if 'class_id' in part:
                class_map = {}
                id_tokens = line.split('class_id =')[1].split(',')
                class_map['class_id'] = int(id_tokens[0].strip())

                    name_tokens = line.split('name =')[1].split(',')
                class_map['class_name'] = name_tokens[0]

                if 'ap =' in line:
                    ap_tokens = line.split('ap =')[1].split(',')
                    class_map['class_ap'] = ap_tokens[0].split('%')[0].strip()

                class_stats.append(class_map)
            elif '(mAP)' in part:
                tokens = line.split('(mAP) =')[1]
                map_stats['mean_avg_precision'] = tokens.split()[2].strip(',')
            elif 'Detection' in part:
                tokens = line.split(':')[1]
                total_detection_time = str(''.join(tokens)).strip()
                map_stats['total_detection_time'] = total_detection_time
    map_stats['class_stats'] = class_stats
print(map_stats,'\n\n')


with open(filepath, 'r') as file:
    line = next(file)
    while line:
        reg_match = _RegExLib(line)

        if reg_match.school:
            school = reg_match.school.group(1)

        if reg_match.grade:
            grade = reg_match.grade.group(1)
            grade = int(grade)

        if reg_match.name_score:
            value_type = reg_match.name_score.group(1)
            line = next(file, None)
            while line and line.strip():
                number, value = line.strip().split(',')
                value = value.strip()
                dict_of_data = {
                    'School': school,
                    'Grade': grade,
                    'Student number': number,
                    value_type: value
                }
                data.append(dict_of_data)
                line = next(file, None)

        line = next(file, None)